In [ ]:
import sys
import math
from heapq import *
from collections import defaultdict
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
sys.setrecursionlimit(10**7)

---
### グラフの作成

In [ ]:
n = 10

# 隣接リストの作成
to = [[] for _ in range(n*n)]
for i in range(n*n):
    if (i+1) % n != 0:
        to[i].append(i+1)
    if i % n != 0:
        to[i].append(i-1)
    if i+n < n*n:
        to[i].append(i+n)
    if i-n >= 0:
        to[i].append(i-n)

In [ ]:
for i in range(n*n):
    print(f'to[{i}] = {to[i]}')

In [ ]:
# 通過不可能領域の生成
def generate_obstruction() -> List:
    """
    通過不可能領域は矩形とする。
    10×10
    """
    area_1 = [12, 13, 22, 23, 32, 33]
    area_2 = [65, 66, 67, 75, 76, 77]
    obs_area_list = area_1 + area_2

    obs_area_dict = defaultdict(int)
    for e in obs_area_list:
        obs_area_dict[e] = 1

    return obs_area_list, obs_area_dict


In [ ]:
# グラフの可視化
def vis_gridpath(n, path_nodes, obs_nodes=None):
    # n×n の格子グラフを作成
    G = nx.grid_2d_graph(n, n)
    mapping = {(i, j): i*n + j for i, j in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
    path_edges = list(zip(path_nodes[:-1], path_nodes[1:]))

    # 描画
    plt.figure(figsize=(6,6))

    # 通常ノード・エッジ
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=500)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_labels(G, pos)

    # 障害物エリアの描画
    if obs_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=obs_nodes, node_color='black', node_size=500)
        labels = {n: n for n in obs_nodes}
        nx.draw_networkx_labels(
            G, pos,
            labels=labels,
            font_color='white',
        )

    # 経路部分を強調（色を変える）
    nx.draw_networkx_nodes(G, pos, nodelist=path_nodes, node_color='orange')
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

    plt.axis('off')
    plt.show()

In [ ]:
# # n×n の格子グラフを作成
# n = 10
# G = nx.grid_2d_graph(n, n)

# mapping = {(i, j): i*n + j for i, j in G.nodes()}
# G = nx.relabel_nodes(G, mapping)

# pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
# nx.draw(G, pos, with_labels=True, node_size=500, node_color='lightblue')
# plt.show()

In [ ]:
obs_nodes_list, obs_nodes_dict = generate_obstruction()

n = 10
path_nodes = [0, 1, 2, 3, 4, 5, 6, 7]
vis_gridpath(n, path_nodes, obs_nodes_list)

In [ ]:
n = 10
path_nodes = [0, 1, 2, 12, 13, 14, 24]
vis_gridpath(n, path_nodes)

In [ ]:
n = 10
path_nodes = []
vis_gridpath(n, path_nodes)

---
### 経路探索

In [ ]:
def dijkstra(start, n, to):
    prevs = [0 for _ in range(n*n)]
    d = [float('inf') for _ in range(n*n)]
    d[start] = 0
    q = [(0, start)]
    while q:
        (ci, i) = heappop(q)
        if d[i] < ci:
            continue
        for j in to[i]:
            if d[j] > d[i] + 1:
                d[j] = d[i] + 1
                prevs[j] = i
                heappush(q, (d[j], j))
    return d, prevs

def get_path(s, v, prevs):
    '''
    s -> vへの最短経路path
    '''
    if v == s:
        return [s]
    path = [v]
    while True:
        path.append(prevs[v])
        v = prevs[v]
        if v == s:
            return path


In [ ]:
d, prevs = dijkstra(0, n*n, to)

path = get_path(0, 66, prevs)
path = path[::-1]

print(path)

In [ ]:
vis_gridpath(10, path)

---
## DFSその1
頂点vからスタートし、長さLで進める経路をすべて列挙する。(anss)
列挙された経路の中で、終点が事前に定めたゴール地点に一致しているものだけを取り出す。

- メリット：
  - 実装が楽
- デメリット：
  - 余計な経路も算出するので計算に時間がかかる。


In [ ]:
used = [False for _ in range(n*n)]
path = []
anss = []

limit = 18
goal = 66


def dfs(v):
    path.append(v)
    used[v] = True    
    if len(path) >= limit+1:
        anss.append(path.copy())
        path.remove(v)
        used[v] = False
        return 
    for u in to[v]:
        if not used[u]:
            dfs(u)
    path.remove(v)
    used[v] = False


In [ ]:
used = [False for _ in range(n*n)]
path = []
anss = []
dfs(0)

print('len(path) =', len(path))
print('len(anss) =', len(anss))


In [ ]:
path = anss[0]

print('len(path) =', len(path))
print('path =', path)
vis_gridpath(10, path)

In [ ]:
anss_goal_66 = []
for path in anss:
    if path[-1] == 66:
        anss_goal_66.append(path)

In [ ]:
len(anss_goal_66)

In [ ]:
path = anss_goal_66[1000]

vis_gridpath(10, path)

In [ ]:
# # 交叉回数
# def crosscount(path1, path2):
#     common = set(path1) & set(path2)
#     return len(common)

---
## DFSその2
頂点vからスタートし、事前に決めたゴール地点に長さLで進める経路をすべて列挙する。


In [ ]:
used = [False for _ in range(n*n)]
path = []
anss = []

limit = 18
goal = 66

def dfs_2(v):
    path.append(v)
    used[v] = True    
    if len(path) >= limit+1:
        if path[-1] == goal:
            anss.append(path.copy())
        path.remove(v)
        used[v] = False
        return 
    for u in to[v]:
        if not used[u]:
            dfs_2(u)
    path.remove(v)
    used[v] = False


# =======================================
used = [False for _ in range(n*n)]
path = []
anss = []
dfs_2(0)

print('len(anss) =', len(anss))



In [ ]:
vis_gridpath(10, anss[4])

In [ ]:
for i in range(12):
    print('i =', i)
    vis_gridpath(10, anss[i])

---
## DFSその3(メモ化再帰、動的計画法)
頂点vからスタートし、事前に決めたゴール地点に長さLで進める経路をすべて列挙する。


In [ ]:
# used = [False for _ in range(n*n)]
# path = []
# anss = []

# limit = 18
# goal = 66

# memo = {}

# def dfs_memo(v):
#     path.append(v)
#     used[v] = True    
#     if len(path) >= limit+1:
#         if path[-1] == goal:
#             anss.append(path.copy())
#         path.remove(v)
#         used[v] = False
#         return 
#     for u in to[v]:
#         if not used[u]:
#             dfs_2(u)
#     path.remove(v)
#     used[v] = False


In [ ]:
used = [False for _ in range(n * n)]
path = []
anss = []

limit = 18
goal = 66
memo = set()  # メモ化用セット（タプルで管理）


def dfs_memo(v, remaining_steps):
    # メモされた探索済み状態ならスキップ
    if (v, remaining_steps) in memo:
        return

    # 終了条件
    path.append(v)
    used[v] = True

    if remaining_steps == 0:
        if v == goal:
            anss.append(path.copy())
        path.pop()
        used[v] = False
        return

    for u in to[v]:
        if not used[u]:
            dfs_memo(u, remaining_steps - 1)

    # バックトラックとメモ記録
    path.pop()
    used[v] = False
    memo.add((v, remaining_steps))

dfs_memo(0, limit)

In [ ]:
used = [False for _ in range(n*n)]
path = []
anss = []

limit = 18
goal = 66

memo = set()

dfs_memo(0, limit)

print('len(path) =', len(path))
print('len(anss) =', len(anss))


In [ ]:
vis_gridpath(10, anss[2])

In [ ]:
used = [False for _ in range(n * n)]
path = []
anss = []

limit = 18
goal = 66
memo = set()  # (v, remaining_steps, used_bitmask)


def dfs_memo_2(v, remaining_steps):
    # used 配列をビットマスクに変換
    used_key = tuple(used)
    key = (v, remaining_steps, used_key)
    if key in memo:
        return

    path.append(v)
    used[v] = True

    if remaining_steps == 0:
        if v == goal:
            anss.append(path.copy())
        path.pop()
        used[v] = False
        return

    for u in to[v]:
        if not used[u]:
            dfs_memo_2(u, remaining_steps - 1)

    path.pop()
    used[v] = False
    memo.add(key)




used = [False for _ in range(n * n)]
path = []
anss = []

limit = 18
goal = 66
memo = set()  # (v, remaining_steps, used_bitmask)
dfs_memo_2(0, limit)


print('len(path) =', len(path))
print('len(anss) =', len(anss))


In [ ]:
vis_gridpath(10, anss[2])

---
## A*を改変

In [ ]:
def astar(start, goal, n, to, h):
    """
    A* アルゴリズムによって、始点 start から終点 goal までの最短距離を求める。
    各エッジのコストは 1 と仮定している。

    Args:
        start (int): 探索の始点ノード（0 以上 n*n 未満）
        goal (int): 探索の終点ノード（0 以上 n*n 未満）
        n (int): グリッドやノード配列の一辺の長さ（n*n がノード数）
        to (list[list[int]]): 隣接リスト。to[i] はノード i に接続するノードのリスト
        h (list[float] または function): ヒューリスティック関数。各ノードに対するゴールまでの推定コスト。
                                          配列または関数形式で指定可能。

    Returns:
        d (list[float]): 各ノードへの実際の最短距離 d[i]
        prevs (list[int]): 各ノード i に到達する直前のノード prevs[i]（経路復元用）
    """
    prevs = [0 for _ in range(n * n)]  # 経路復元のための直前ノード記録
    d = [float('inf') for _ in range(n * n)]  # 各ノードまでの最短距離（g値）
    d[start] = 0  # 始点の距離は0

    # 優先度付きキューには (f値 = g + h, ノード番号) を格納
    initial_h = h[start] if isinstance(h, list) else h(start)
    q = [(initial_h, start)]

    while q:
        f, i = heappop(q)  # f = 推定総コスト, i = 現在のノード

        if i == goal:
            break  # ゴールに到達したら終了

        # キューに古い（非最短）情報が残っていた場合はスキップ
        current_h = h[i] if isinstance(h, list) else h(i)
        if d[i] + current_h < f:
            continue

        # 隣接ノード j について緩和処理
        for j in to[i]:
            tentative_g = d[i] + 1  # エッジコストは常に1
            if d[j] > tentative_g:
                d[j] = tentative_g
                prevs[j] = i
                f_j = d[j] + (h[j] if isinstance(h, list) else h(j))  # f = g + h
                heappush(q, (f_j, j))

    return d, prevs

In [ ]:
def euclid_dist(i, j):
    '''
    頂点iと頂点jの間のeuclid距離
    '''
    i_h = i%n
    i_v = i//n
    j_h = j%n
    j_v = j//n
    d_h = abs(i_h - j_h)
    d_v = abs(i_v - j_v)
    d = math.sqrt(d_h*d_h + d_v*d_v)
    return d


def manhattan_dist(i, j):
    '''
    頂点iと頂点jの間のmanhattan距離
    '''
    i_h = i%n
    i_v = i//n
    j_h = j%n
    j_v = j//n
    d_h = abs(i_h - j_h)
    d_v = abs(i_v - j_v)
    d = d_h + d_v
    return d


def heuristic(v):
    return euclid_dist(v, n)
    # return manhattan_dist(v, n)


In [ ]:
d, prevs = astar(0, 66, 10, to, heuristic)

path = get_path(0, 66, prevs)
path = path[::-1]

In [ ]:
vis_gridpath(10, path)

In [ ]:
def astar_deformed(start, goal, n, to, h, L):
    """
    変形されたA*
    """
    prevs = [None for _ in range(n * n)]
    visited = [False for _ in range(n * n)]
    
    # (f = g + h, g, ノード番号)
    initial_h = h[start] if isinstance(h, list) else h(start)
    q = [(initial_h, 0, start)]
    
    while q:
        f, g, i = heappop(q)

        # 既にこのノードを g ステップ以内で訪れていたらスキップ
        if visited[i] and g >= L:
            continue
        visited[i] = True

        # 成功条件：g == L かつ ゴール到達
        if i == goal and g == L:
            break
        if g >= L:
            continue  # これ以上進めない

        for j in to[i]:
            tentative_g = g + 1
            if tentative_g <= L:
                f_j = tentative_g + (h[j] if isinstance(h, list) else h(j))
                heappush(q, (f_j, tentative_g, j))
                prevs[j] = i

    return prevs

In [ ]:
# d, prevs = astar_deformed(0, 66, 10, to, heuristic, 18)

# path = get_path(0, 66, prevs)
# path = path[::-1]

In [ ]:
print('d[66] =', d[66])
vis_gridpath(10, path)

現状の結論

DFS1　〇
DFS2　◎
DFS3　×　バグっててうまく答え出ない。（2種類のコードをChatGPTに出してもらったが。。。）

---
## GAの練習

In [ ]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.termination import get_termination

# 1. 問題の定義
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=2, n_obj=1, n_constr=0, xl=np.array([-5, -5]), xu=np.array([5, 5]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[0]**2 + x[1]**2
        out["F"] = f1

# 2. 問題のインスタンスを作成
problem = MyProblem()

# 3. アルゴリズムの選択と設定
algorithm = GA(pop_size=20, eliminate_duplicates=True) # 各世代で20個体(解)を保持する。

# 4. 最適化の実行
termination = get_termination("n_gen", 100) # 100世代まわす
res = minimize(problem, algorithm, termination, seed=1, verbose=True)

# 5. 結果の表示
print("最適化された設計変数: %s" % res.X)
print("最小化された目的関数値: %s" % res.F)

### TSPをGAで解く

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
# from pymoo.operators.sampling.perm_random import PermutationRandomSampling
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.optimize import minimize

# 都市数
N = 20

# 各都市の座標（ランダムに生成）
np.random.seed(42)
cities = np.random.rand(N, 2)

# 巡回セールスマン問題を定義
class TSP(ElementwiseProblem):
    def __init__(self, cities):
        super().__init__(
            n_var = len(cities),
            n_obj = 1,
            n_constr = 0,
            xl = 0,
            xu = len(cities)-1,
            type_var = np.int64,
            elementwise_evaluation = True
        )
        self.cities = cities

    def _evaluate(self, x, out, *args, **kwargs):
        '''
        Arg:
            x : 決定変数。今の場合、xは「都市の訪問順を表す長さNの順列」を表す。
        '''
        # 巡回経路の距離を計算
        dist = 0.0
        for i in range(len(x)):
            a = self.cities[x[i]]
            b = self.cities[x[(i+1) % len(x)]]
            dist += np.linalg.norm(a-b)
        out['F'] = dist

problem = TSP(cities)

# GAアルゴリズムの構成（順列用）
algorithm = GA(
    pop_size = 100, # 集団サイズ（個体数）
    sampling = PermutationRandomSampling(), # 都市の訪問順列の初期サンプリング
    crossover = OrderCrossover(), # TSPに適した交叉（順序保持型交叉）
    mutation = InversionMutation(), # 順列に対する突然変異オペレータ
    eliminate_duplicates=True, # 重複排除。個体ごとに評価を行う設定（順列にはこれが必要） 
)

# 最適化の実行
res = minimize(
    problem,
    algorithm,
    termination=('n_gen', 200),  # 200世代
    seed=1,
    verbose=True
)

# 最良解の表示
print("Best tour:", res.X)
print("Best distance:", res.F[0])


# 結果の可視化
def plot_tour(order, cities):
    path = cities[order]
    path = np.vstack([path, path[0]])  # 巡回するために始点を最後に加える
    plt.plot(path[:, 0], path[:, 1], marker='o')
    plt.title("Best TSP Tour")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()


plot_tour(res.X, cities)
